<a href="https://colab.research.google.com/github/gamidirohan/MachineLearning-Lab/blob/main/Lab07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A2.

Importing necessary libraries

In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import pickle
import os

Class Labels

In [27]:
class_labels = ["acrostic", "ballad", "epigram", "haiku", "limerick", "sestina", "sonnet", "villanelle"]

Load Dataset from .csv file

In [28]:
def load_data(dataset_file):
    data_df = pd.read_csv(dataset_file)
    X = data_df.drop(columns=['label']).values
    y = data_df['label'].values
    return X, y

Evaluate model

In [29]:
def evaluate_model(model, X_train, X_test, y_train, y_test, class_labels):
    train_accuracy = model.score(X_train, y_train)
    test_accuracy = model.score(X_test, y_test)
    print(f"Train Accuracy: {train_accuracy:.2f}")
    print(f"Test Accuracy: {test_accuracy:.2f}")

    # Generate classification report
    y_pred = model.predict(X_test)
    report = classification_report(y_test, model.predict(X_test), target_names=class_labels, zero_division=1)
    print("Classification Report:")
    print(report)

Save model as a .pkl file

In [30]:
def save_model(model, model_file):
    with open(model_file, 'wb') as f:
        pickle.dump(model, f)
    print(f"Model saved as {model_file}")

Load Dataset

In [31]:
import pandas as pd

# Load the dataset into a DataFrame
data_df = pd.read_csv("poems_data.csv")

# Drop rows with missing values
data_df.dropna(inplace=True)

# Extract features and target variable
X = data_df.drop(columns=['label']).values
y = data_df['label'].values

Splitting data into Train and Test datasets

In [32]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Parameter Grid for MLP

In [33]:
mlp_param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (150,), (200,)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1]
}

RandomSearchCV for MLP

In [34]:
mlp_random_search = RandomizedSearchCV(
    estimator=MLPClassifier(),
    param_distributions=mlp_param_grid,
    n_iter=10,
    scoring='accuracy',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)
mlp_random_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=MLPClassifier(), n_jobs=-1,
                   param_distributions={'activation': ['relu', 'tanh'],
                                        'hidden_layer_sizes': [(50,), (100,),
                                                               (150,), (200,)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'learning_rate_init': [0.001, 0.01,
                                                               0.1],
                                        'solver': ['adam', 'sgd']},
                   random_state=42, scoring='accuracy', verbose=2)

Print MLP best Parameters

In [35]:
print("Best parameters for MLP:")
print(mlp_random_search.best_params_)

Best parameters for MLP:
{'solver': 'adam', 'learning_rate_init': 0.001, 'learning_rate': 'constant', 'hidden_layer_sizes': (50,), 'activation': 'relu'}


Evaluate MLP with best parameters

In [36]:
mlp_model = mlp_random_search.best_estimator_
print("Evaluating MLP...")
evaluate_model(mlp_model, X_train, X_test, y_train, y_test, class_labels)

Evaluating MLP...
Train Accuracy: 0.99
Test Accuracy: 0.81
Classification Report:
              precision    recall  f1-score   support

    acrostic       0.90      0.67      0.77        27
      ballad       0.85      0.81      0.83        21
     epigram       0.79      0.86      0.83        22
       haiku       0.75      0.84      0.79        25
    limerick       1.00      0.93      0.96        14
     sestina       0.87      0.72      0.79        18
      sonnet       0.67      1.00      0.80        10
  villanelle       0.57      0.80      0.67         5

    accuracy                           0.81       142
   macro avg       0.80      0.83      0.80       142
weighted avg       0.83      0.81      0.81       142



Save MLP model as .pkl

In [37]:
mlp_model_file = "mlp_model.pkl"
save_model(mlp_model, mlp_model_file)

Model saved as mlp_model.pkl


Defining parameter grid for Perceptron

In [38]:
perceptron_param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'max_iter': [1000, 2000, 3000],
    'tol': [1e-3, 1e-4, 1e-5]
}

Perform RandsearchCV for Perceptron

In [39]:
perceptron_random_search = RandomizedSearchCV(
    estimator=Perceptron(),
    param_distributions=perceptron_param_grid,
    n_iter=10,
    scoring='accuracy',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)
perceptron_random_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=Perceptron(), n_jobs=-1,
                   param_distributions={'alpha': [0.0001, 0.001, 0.01, 0.1],
                                        'max_iter': [1000, 2000, 3000],
                                        'tol': [0.001, 0.0001, 1e-05]},
                   random_state=42, scoring='accuracy', verbose=2)

Printing best Perceptron Parameters

In [40]:
print("Best parameters for Perceptron:")
print(perceptron_random_search.best_params_)

Best parameters for Perceptron:
{'tol': 1e-05, 'max_iter': 3000, 'alpha': 0.1}


Evaluate Perceptron with best Parameters

In [41]:
perceptron_model = perceptron_random_search.best_estimator_
print("Evaluating Perceptron...")
evaluate_model(perceptron_model, X_train, X_test, y_train, y_test, class_labels)

Evaluating Perceptron...
Train Accuracy: 0.27
Test Accuracy: 0.36
Classification Report:
              precision    recall  f1-score   support

    acrostic       0.27      0.93      0.42        27
      ballad       0.50      0.05      0.09        21
     epigram       0.45      0.23      0.30        22
       haiku       0.55      0.72      0.62        25
    limerick       1.00      0.00      0.00        14
     sestina       1.00      0.00      0.00        18
      sonnet       1.00      0.00      0.00        10
  villanelle       0.50      0.40      0.44         5

    accuracy                           0.36       142
   macro avg       0.66      0.29      0.23       142
weighted avg       0.61      0.36      0.26       142



Save model as .pkl

In [42]:
perceptron_model_file = "perceptron_model.pkl"
save_model(perceptron_model, perceptron_model_file)

Model saved as perceptron_model.pkl
